In [ ]:
import sys
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.multi_sim import MultiSim
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.gps_importer import *
from Functions.py_functions.las_solvers.octahedral import Octahedral_LAS
from Functions.py_functions.las_solvers.multi_layer import Multi_Layer_LAS

In [ ]:
tracks = []
# tracks = []
v_average = 15
mu_corr = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
las = Multi_Layer_LAS()
las.add_layers = 2

In [ ]:
# here we make a function that generates a car with a given roll stiffness distribution and chassis roll stiffness
# the first and second arguments take the values of the first and second sweep variables that you give to run_sim
def gen_car_lltd(rsd, k_c):
    total_roll_stiffness = 900 * FTLB_TO_NM
    car = Car(front_axle_weight=0.4725)
    car.z_f = 0.13 * IN_TO_M
    car.z_r = 1.0 * IN_TO_M
    car.cdA = 1.33
    car.clA = 3.43
    car.cg_height=10.7*IN_TO_M
    car.front_axle_downforce = 0.5
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.k_c = 1000000
    car.update_car()
    car.description = f"rsd:{rsd:.2f} k_c:{k_c:.2f}"
    return car
# do a sweep of roll stiffness distribution and chassis roll stiffness
rsd_range = np.linspace(0.25, 0.75, 10)
k_c_r = np.linspace(500, 2500, 1) # np.array([1000]) #
sim = MultiSim(tracks, gen_car_lltd, rsd_range, k_c_r, "Roll Stiffness Distribution", "Chassis Roll Stiffness (lbf/deg)")

In [ ]:
sim.run_lltd()
sim.run_sim(las)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
fig4 = go.Figure()
for ind in range(len(sim.lat_acc)):
    index = sim.inds[ind]
    lltd = sim.lltd[index[0], index[1]]
    # fig4.add_trace(go.Scatter(x=sim.pos_vel[ind], y=sim.lat_acc[ind][:, 1], text=sim.pos_name[ind], marker_color=sim.pos_vel[ind], legendgroup=f"group{ind}", showlegend=False), row=1, col=1)
    fig4.add_trace(go.Scatter(x=sim.pos_vel[ind] * MS_TO_MPH, y=sim.lat_acc[ind][:, 2], mode="lines", legendgroup=f"group{ind}", showlegend=True, name=f"LLTD: {lltd:.2f}"))

fig4.update_yaxes(title_text='Yaw Acc (rad/s^2)')
fig4.update_xaxes(title_text='Velocity (mph)')
# fig4.update_layout(template="plotly_dark") , title_x=0.5, legend_title_text='Track', legend_title_font_size=18, legend_font_size=16, font_size=18
fig4.update_layout(title_text="Yaw Acc vs LLTD", width=1000, height=600)
fig4.show()